# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer some test questions

In [1]:
# Clone the git repo to access the utilities
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 184 (delta 0), reused 0 (delta 0), pack-reused 183 (from 3)
Receiving objects: 100% (184/184), 616.68 MiB | 43.34 MiB/s, done.
Resolving deltas: 100% (94/94), done.
Updating files: 100% (53/53), done.


In [2]:
# Make sure to be on the repo directory and pull
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git fetch --all
!git reset --hard origin/main

/kaggle/working/mistralpeft
Fetching origin
HEAD is now at a3248d8 more results


In [3]:
# Install the required packages from pypi
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.5 MB/s eta 0:00:00


In [4]:
# Load packages
from transformers import Trainer, TrainingArguments, AutoTokenizer, pipeline
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel
from datasets import Dataset

from mistralpeft.utils import TextExtractor, CLMPreprocessor

In [5]:
# Login to HuggingFace using Kaggle's secrets to be able to download models
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF
Several PDFs from my former research group at university (Thermo-Fluiddynamics Group, Prof. Polifke) are chosen to form the corpus

In [6]:
# TextExtractor is a simple ETL class to acquire a text corpus
pdf_files = [
    "Dissertation.pdf",
]
    
pdf_urls = [
    "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    "https://mediatum.ub.tum.de/doc/1597610/1597610.pdf"
    "https://mediatum.ub.tum.de/doc/1584750/1584750.pdf",
    "https://mediatum.ub.tum.de/doc/1484812/1484812.pdf",
    "https://mediatum.ub.tum.de/doc/1335646/1335646.pdf",
    "https://mediatum.ub.tum.de/doc/1326486/1326486.pdf",
    "https://mediatum.ub.tum.de/doc/1306410/1306410.pdf",
    "https://mediatum.ub.tum.de/doc/1444929/1444929.pdf",
]

data_path = Path("data/processed_documents.json")
if not data_path.is_file():
    with TextExtractor("data/processed_documents.json") as extractor:
        # Process local files
        extractor.process_documents(pdf_files)
            
        # Process URLs
        extractor.process_documents(pdf_urls, url_list=True)

## 2. Prepare MCLM Training Data

In [7]:
# Specify the model and load the tokenizer
# Llama 3.2 3B with approx. 3 billion parameters
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [8]:
# Preprocess the corpus for Causal Language Modeling (CLM)
json_file_paths = ["data/processed_documents.json"]
preprocessor = CLMPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [9]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 232 training examples and 26 test examples

Example input:
, a model which utilizes a low Mach number, reactive ﬂow simulation. Here, the density depends only on the tem- perature, but not on pressure. As shown in Fig. 2, this model is coupled to a low-order network model via a reference velocity and the global heat release rate. Con- sequently, we denote this model “LM-uq”. Please note that in the literature [11–14] other hy- brid models for thermoacoustic oscillations have been proposed. However, the coupling used by these hy- brid models has not been cross-validated in a systematic manner against a fully compressible simulation. This cross-validation allows to directly verify the coupling between ﬂame, hydrodynamics and acoustics. In the following section the two formulations are ex- plained in detail. Thereafter, in Section 3 the results of the two models are compared via a bifurcation analy- sis. Although, complex thermoacoustic oscillations are observed, the two models ar

## 3. Load and Prepare Model

In [10]:
# Load the base model
# Q4_K_M quantization of the base model is achieved through BitsAndBytes. It requires CUDA!
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors for Q4_K_M
    )

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
    )
if base_model.config.pad_token_id is None:
    base_model.config.pad_token_id = base_model.config.eos_token_id

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [11]:
# Configure the (Q)LoRA adaptor to use a rank of r=4
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)

## 4. Train the Model

The LoRA adapter has about 6M parameters to train (compared to 3B parameters of the full LLM)

In [12]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3, # Creates a virtual batch size of 3
    learning_rate=1e-4,
    fp16=True, # numerical precision of adapter is float16
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit", # Memory efficient optimizer
    log_level="info",
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train(resume_from_checkpoint="results/checkpoint-312")

Using auto half precision backend
Loading model from results/checkpoint-312.
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this expe

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=312, training_loss=0.0, metrics={'train_runtime': 0.1427, 'train_samples_per_second': 6505.351, 'train_steps_per_second': 2159.104, 'total_flos': 6.484035595822694e+16, 'train_loss': 0.0, 'epoch': 4.0})

In [13]:
# Save the LoRA adapter weights:
lora_save_path = "lora_weights" 
peft_model.save_pretrained(lora_save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "u

## 6. Test the Model

peft_model = PeftModel.from_pretrained(base_model, "results/checkpoint-231")

In [14]:
peft_model = PeftModel.from_pretrained(base_model, "results/checkpoint-234")

In [15]:
def test_continuation(model, tokenizer):# Example queries
    queries = [
        "SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to",
        "In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities.",
        "The unit impulse response is a time domain model. It shows",
    ]
    
    # Generate responses
    for query in queries:
            
        inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True).to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)
        
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\nQuery: {query}")
        print(f"\nResponse: {text}")
        print("-" * 80)

In [16]:
### 6.1 Answers to the test questions by the instruct model w/o LoRA

### 6.2 Answers to the test questions by the adapted model

In [17]:
test_continuation(base_model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to

Response: SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to the the between values the of quantities interest the. The of is to the the and the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities.

Response: In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities. mechanisms this. study the of transfer puls heat oscill flow inst inacur heat in pul flow of puls heat flow heat transfer puls flow puls oscill flow puls heat heat puls transfer puls flow oscill heat puls flow heat puls flow transfer heat puls flow heat transfer puls heat oscill heat transfer heat oscill puls flow heat puls flow transfer puls oscill flow transfer oscill heat heat puls transfer puls flow heat transfer flow puls oscill puls flow transfer heat oscill transfer heat puls transfer oscill heat transfer heat flow transfer puls oscill puls transfer heat puls oscill heat puls flow transfer oscill flow heat flow puls oscill transfer heat oscill heat transfer heat puls oscill transfer puls flow heat flow oscill transfer oscill heat transfer flow oscill heat flow transfer puls heat flow pul

In [18]:
test_continuation(peft_model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to

Response: SI is used when processes are either too complex to gain insight using first principles, i.e. physical laws, or the calculation is too costly in terms of time or resources. Its goal is to the the of the by the of the. is a of. of of. of. is a. is................................................................................................................................
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities.

Response: In pulsating or oscillating flow, heat transfer can damp, but also drive instabilities. present work a onoretical of two different that the heat in puls flow a heat in puls oscill flow heat puls oscill flow heat puls flow heat puls flow puls flow oscill heat oscill heat heat oscill heat oscill flow heat flow flow heat heat flow flow flow heat flow oscill flow oscill heat flow heat flow oscill oscill flow heat heat flow flow flow heat oscill heat oscill flow heat flow flow heat flow flow heat flow flow heat flow heat flow heat flow heat flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow
------------------------------------